In [ ]:
# API_KEY = 


In [5]:
import google.generativeai as genai
import os

# Replace 'your-api-key-here' with your actual API key for GenAI
API_KEY = "your-api-key-here" # Ayushi, I replaced this so nobody sees your API key :) 

# Initialize the GenAI client
genai.configure(api_key=API_KEY)

In [8]:
import time
import json

base_model = "models/gemini-1.5-flash-001-tuning"


with open("/Users/ayushimathur/Downloads/cs6220-project-eval_will/gemini_finetuning/training_data.json") as f:
    training_data = json.load(f)

truncated_training_data = []

for _ in training_data:
    if len(_["output"]) > 2000:
        continue
    else:
        truncated_training_data.append(_)

operation = genai.create_tuned_model(
    # You can use a tuned model here too. Set `source_model="tunedModels/..."`
    display_name="increment",
    source_model=base_model,
    epoch_count=20,
    batch_size=4,
    learning_rate=0.001,
    training_data=truncated_training_data,
    
)

for status in operation.wait_bar():
    time.sleep(10)

result = operation.result()
print(result)
# # You can plot the loss curve with:
# snapshots = pd.DataFrame(result.tuning_task.snapshots)
# sns.lineplot(data=snapshots, x='epoch', y='mean_loss')

model = genai.GenerativeModel(model_name=result.name)
result = model.generate_content("III")
print(result.text)  # IV

  0%|          | 0/2320 [00:00<?, ?it/s]

TunedModel(name='tunedModels/increment-x0nduqe11yoh',
           source_model='models/gemini-1.5-flash-001-tuning',
           base_model='models/gemini-1.5-flash-001-tuning',
           display_name='increment',
           description='',
           temperature=1.0,
           top_p=0.95,
           top_k=64,
           state=<State.ACTIVE: 2>,
           create_time=datetime.datetime(2024, 11, 25, 12, 35, 50, 752843, tzinfo=datetime.timezone.utc),
           update_time=datetime.datetime(2024, 11, 25, 14, 10, 41, 894410, tzinfo=datetime.timezone.utc),
           tuning_task=TuningTask(start_time=datetime.datetime(2024, 11, 25, 12, 35, 55, 521090, tzinfo=datetime.timezone.utc),
                                  complete_time=datetime.datetime(2024, 11, 25, 14, 10, 41, 894410, tzinfo=datetime.timezone.utc),
                                  snapshots=[...],
                                  hyperparameters=Hyperparameters(epoch_count=20,
                                                

In [9]:
print(result)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "What can I help you with? Please provide me with more context or a specific question. For example, you could ask:\n\n* \"What is the capital of France?\"\n* \"Can you write a short poem about the ocean?\"\n* \"What is the meaning of life?\"\n\nI'm ready to assist you!"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
             

In [16]:
# Testing the model
import time
from tqdm import tqdm
testing_outputs = []
with open("/Users/ayushimathur/Downloads/cs6220-project-eval_will/gemini_finetuning/testing_data.json") as f:
    testing_data = json.load(f)
for idx, _ in tqdm(enumerate(testing_data)):
    try:
        result = model.generate_content(_["text_input"])
        result = result.text
    except:
        result = ""
    testing_output = {
        "prompt": _["text_input"],
        "gold_ans": _["output"],
        "response": result
    }
    testing_outputs.append(testing_output)
    if idx%500 == 499:
        time.sleep(100)

with open("/Users/ayushimathur/Downloads/cs6220-project-eval_will/gemini_finetuning/model_response.json", "w") as f:
    json.dump(testing_outputs, f)

710it [07:53,  1.50it/s]
